In [1]:

using PowerModels
using JuMP
using Ipopt

In [2]:
const PMs =PowerModels

PowerModels

In [7]:
function minimize_load_shed(data,solver)
    
    # extract the reference for the data
    ref = PMs.build_ref(data)[:nw][0]
    model = Model(with_optimizer(solver))

    
    load_index=Int64[]
    for(i,load) in ref[:load]
        push!(load_index,load["load_bus"])
    end
    
    gen_index=Int64[]
    for(i,gen) in ref[:gen]
        #print(gen)
        push!(gen_index,gen["gen_bus"])
    end
    
    branch_count = length(ref[:branch])
    bus_count = length(ref[:bus])
    gen_count = length(ref[:gen])
    load_count = length(ref[:load])
    
    @variables(model,begin
            p_k[i=1:branch_count]
            trans_k[i=1:branch_count]
            p_g[i=1:bus_count]
            p_l[i=1:bus_count]
            p_b_LS[i=1:bus_count]>=0
            theta_b[i=1:bus_count]
            end)
    
    # define objective function.. minimization of load shedding
    @objective(model, Min, sum(p_b_LS[i] for i in 1:bus_count))
    
    # voltage angle at all bus within -pi to +pi
    for i in 1:bus_count
       @constraint(model, theta_b[i] <= pi)
       @constraint(model, theta_b[i] >= -pi)
    end
    
    #flow equality constraint
    for (i,branch) in ref[:branch]
        from_index = branch["f_bus"]
        to_index = branch["t_bus"]
        @constraint(model, p_k[i] == branch["b_fr"]*(theta_b[from_index] - theta_b[to_index]))
    end
    
    #branch limits
    for (i,branch) in ref[:branch]
       @constraint(model, p_k[i] <= branch["rateA"][i])
       @constraint(model, p_k[i] >= -branch["rateA"][i])
    end
    
    # generator limit constraints
    for (i,gen) in ref[:gen]
        @constraint(model, p_g[gen["gen_bus"]] <= gen["pmax"])
        @constraint(model, p_g[gen["gen_bus"]] >= gen["pmin"])
    end
    
    # load shedding constraint
    for (i,load) in ref[:load]
        @constraint(model,p_b_LS[load["load_bus"]] <= load["pd"])
        @constraint(model,p_l[load["load_bus"]] == load["pd"])
    end
    
    # set p_g zero for all non-gen bus
    for i in 1:bus_count
        if (!(i in gen_index))
            @constraint(model,p_g[i]==0)
        end
    end
    
    # set p_l zero for all non-gen bus
    for i in 1:bus_count
        if (!(i in load_index))
            @constraint(model,p_l[i]==0)
            @constraint(model,p_b_LS[i]==0)
        end
    end
    
    # power balance constraint
    for (i,bus) in ref[:bus]
        out_branch =[]
        in_branch =[]
        for (j,branch) in ref[:branch]
            if (branch["f_bus"] == bus["index"])
                push!(out_branch,branch["f_bus"])
            elseif (branch["t_bus"] == bus["index"])
                push!(in_branch,branch["t_bus"])
            end
        end
        @constraint(model, (p_g[i] - sum(p_k[k] for k in out_branch) + sum(p_k[j] for j in in_branch))
            <= (p_l[i] - p_b_LS[i]) )
    end

     # Solve statement
    optimize!(model)
    
    return(
        soln =value.(p_b_LS),
        new_gen =value.(p_g),
        theta =value.(theta_b),
        line_flow =value.(p_k),
        load =value.(p_l),
        loadshed = objective_value(model)
    )
end

minimize_load_shed (generic function with 1 method)

In [8]:
#main program 
case_data = PowerModels.parse_file("matpower/case14.m")
nlp_solver = with_optimizer(Ipopt.Optimizer, print_level=0)
#res = minimize_load_shed(case_data, nlp_solver)

for (i,gen) in case_data["gen"]
        # remove the generator
        gen["gen_status"] = 0
        println("gen $(i) out:")
        res = minimize_load_shed(case_data, nlp_solver)
        
        println("Total Load Shed: \$", res.loadshed)
        println(" Load shed: ", res.soln, " MW")
        println("New Gen: ", res.new_gen, "MW")
        println("Theta: ", res.theta)
        println("Line Flow: ", res.line_flow)
        println("Load: ", res.load)
        # restore the generator
        gen["gen_status"] = 1
end

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 12 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 12 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 20 from -360.0 to -60.0 deg.
[w

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = minimize_load_shed(::Dict{String,Any}, ::MathOptInterface.OptimizerWithAttributes) at In[7]:5
└ @ Main .\In[7]:5


KeyError: KeyError: key "rateA" not found

In [17]:
#tnep
function build_tnep(pm::AbstractPowerModel)
    println("Abhijeet : build_tnep")
    variable_bus_voltage(pm)
    variable_gen_power(pm)
    variable_branch_power(pm)
    variable_dcline_power(pm)

    variable_ne_branch_indicator(pm)
    variable_ne_branch_power(pm)
    variable_ne_branch_voltage(pm)

    objective_tnep_cost(pm)

    constraint_model_voltage(pm)
    constraint_ne_model_voltage(pm)

    for i in ids(pm, :ref_buses)
        constraint_theta_ref(pm, i)
    end

    for i in ids(pm, :bus)
        constraint_ne_power_balance(pm, i)
    end

    for i in ids(pm, :branch)
        constraint_ohms_yt_from(pm, i)
        constraint_ohms_yt_to(pm, i)

        constraint_voltage_angle_difference(pm, i)

        constraint_thermal_limit_from(pm, i)
        constraint_thermal_limit_to(pm, i)
    end

    for i in ids(pm, :ne_branch)
        constraint_ne_ohms_yt_from(pm, i)
        constraint_ne_ohms_yt_to(pm, i)

        constraint_ne_voltage_angle_difference(pm, i)

        constraint_ne_thermal_limit_from(pm, i)
        constraint_ne_thermal_limit_to(pm, i)
    end

    for i in ids(pm, :dcline)
        constraint_dcline_power_losses(pm, i)
    end
end


build_tnep (generic function with 1 method)

In [18]:
"Cost of building branches"
function objective_tnep_cost(pm::AbstractPowerModel)
    println("Abhijeet : objective_tnep_cost")
    return JuMP.@objective(pm.model, Min,
        sum(
            sum( branch["construction_cost"]*var(pm, n, :branch_ne, i) for (i,branch) in nw_ref[:ne_branch] )
        for (n, nw_ref) in nws(pm))
    )
end

objective_tnep_cost

In [19]:
function ref_add_ne_branch!(ref::Dict{Symbol,<:Any}, data::Dict{String,<:Any})
    println("Abhijeet : ref_add_ne_branch")
    for (nw, nw_ref) in ref[:nw]
        if !haskey(nw_ref, :ne_branch)
            error(_LOGGER, "required ne_branch data not found")
        end

        nw_ref[:ne_branch] = Dict(x for x in nw_ref[:ne_branch] if (x.second["br_status"] == 1 && x.second["f_bus"] in keys(nw_ref[:bus]) && x.second["t_bus"] in keys(nw_ref[:bus])))

        nw_ref[:ne_arcs_from] = [(i,branch["f_bus"],branch["t_bus"]) for (i,branch) in nw_ref[:ne_branch]]
        nw_ref[:ne_arcs_to]   = [(i,branch["t_bus"],branch["f_bus"]) for (i,branch) in nw_ref[:ne_branch]]
        nw_ref[:ne_arcs] = [nw_ref[:ne_arcs_from]; nw_ref[:ne_arcs_to]]

        ne_bus_arcs = Dict((i, []) for (i,bus) in nw_ref[:bus])
        for (l,i,j) in nw_ref[:ne_arcs]
            push!(ne_bus_arcs[i], (l,i,j))
        end
        nw_ref[:ne_bus_arcs] = ne_bus_arcs

        if !haskey(nw_ref, :ne_buspairs)
            ismc = haskey(nw_ref, :conductors)
            cid = nw_ref[:conductor_ids]
            nw_ref[:ne_buspairs] = calc_buspair_parameters(nw_ref[:bus], nw_ref[:ne_branch], cid, ismc)
        end
    end
end

ref_add_ne_branch! (generic function with 1 method)

In [24]:
function run_tnep_own(file, model_type::Type, optimizer; kwargs...)
    println("Abhijeet : run_tnep")
    return run_model(file, model_type, optimizer, build_tnep; ref_extensions=[ref_add_on_off_va_bounds!,ref_add_ne_branch!], kwargs...)
end

run_tnep_own (generic function with 1 method)

In [25]:

using Cbc
using Juniper
optimizer = Juniper.Optimizer
nl_solver= optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
mip_solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 0)
solver = optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver, "mip_solver"=>mip_solver)

result = run_tnep_own("matpower/case5_tnep.m", DCMPPowerModel, solver)


Abhijeet : run_tnep
[info | PowerModels]: extending matpower format with data: ne_branch 3x15
[warn | PowerModels]: the dc line model is not meant to be used bi-directionally when loss1 > 0, be careful interpreting the results as the dc line losses can now be negative. change loss1 to 0 to avoid this warning
[warn | PowerModels]: the to bus voltage setpoint on dc line 1 does not match the value at bus 5
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from dcline 1: [4000.0, 0.0]
Abhijeet : ref_add_ne_branch
Abhijeet : build_tnep
Abhijeet : objective_tnep_cost
nl_solver   : MathOptInterface.OptimizerWithAttributes(Ipo

┌ Info: no explicit NLP constraints or objective provided using @NLconstraint or @NLobjective macros
└ @ Juniper C:\Users\abhij\.julia\packages\Juniper\Hm6I1\src\MOI_wrapper\MOI_wrapper.jl:346


       0.8826               0.0          0.0 

FP: 0.07400012016296387 s
FP: 1 round
FP: Obj: 1.0

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    1       2                1.0                         0.12            88.26%   0.0       0         -     
    0       3                1.0                         0.12             87.5%   0.1       -       84.7%   

#branches: 2
Obj: 0.9999999885189191


Dict{String,Any} with 8 entries:
  "solve_time"         => 0.245
  "optimizer"          => "Juniper"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 1.0
  "solution"           => Dict{String,Any}("ne_branch"=>Dict{String,Any}("1"=>D…
  "objective_lb"       => 1.0

In [27]:
println(result["solution"]["ne_branch"]["1"]["built"])
println(result["solution"]["ne_branch"]["2"]["built"])
println(result["objective"])

0.0
1.0
0.9999999885189191


In [31]:
result_ac = run_tnep_own("matpower/case5_tnep.m",ACPPowerModel, solver)
println(result_ac["solution"]["ne_branch"]["1"]["built"])
println(result_ac["solution"]["ne_branch"]["2"]["built"])
println(result_ac["objective"])


Abhijeet : run_tnep
[info | PowerModels]: extending matpower format with data: ne_branch 3x15
[warn | PowerModels]: the dc line model is not meant to be used bi-directionally when loss1 > 0, be careful interpreting the results as the dc line losses can now be negative. change loss1 to 0 to avoid this warning
[warn | PowerModels]: the to bus voltage setpoint on dc line 1 does not match the value at bus 5
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from dcline 1: [4000.0, 0.0]
Abhijeet : ref_add_ne_branch
Abhijeet : build_tnep
Abhijeet : objective_tnep_cost
nl_solver   : MathOptInterface.OptimizerWithAttributes(Ipo